# 勾配消失問題
多層パーセプトロンでは、層の長さを長くすればするほど表現力は増します。一方で、学習が難しくなるという問題が知られています。

In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import utils

中間層が10層という深い多層パーセプトロンを用いて、モデル中の重みパラメータの大きさ、勾配の大きさを調べてみます。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import RandomNormal, RandomUniform
from tensorflow.keras.layers import Input, Dense

# データセットの生成
nSamples = 1000
nFeatures = 50
# 100個の入力変数を持つイベント1000個生成。それぞれの入力変数は正規分布に従う
x = np.random.randn(nSamples, nFeatures)
# 正解ラベルは0 or 1でランダムに生成
t = np.random.randint(2, size=nSamples).reshape([nSamples, 1])

# モデルの定義
# 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
model = Sequential()
model.add(Input(shape=(nFeatures,)))
for _ in range(10):
    model.add(
        Dense(
            units=50,
            # 中間層の各ノードで使う活性化関数
            activation="sigmoid",
            # weight(wij)の初期値。ここでは正規分布に従って初期化する
            kernel_initializer=RandomNormal(mean=0.0, stddev=1.0),
        )
    )
model.add(
    Dense(
        units=1,
        activation="sigmoid",
        # weight(wij)の初期値。ここでは正規分布に従って初期化する
        kernel_initializer=RandomNormal(mean=0.0, stddev=1.0),
    )
)

model.compile(loss="binary_crossentropy", optimizer="adam")

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(3, 1, figsize=(6, 12))

# ウェイト(wij)の初期値をプロット
utils.plot_model_weights(model, ax=ax[0])

# 各ノードの出力(sigma(ai))をプロット
utils.plot_model_hidden_nodes(model, x, ax=ax[1])

# ウェイト(wij)の微分(dE/dwij)をプロット
utils.plot_model_weight_gradients(model, x, t, ax=ax[2])

# 図を表示
plt.show()

上段のプロットはパラメータ($w_{ij}$)の初期値を表しています。指定したとおり、各層で正規分布に従って初期化されています。

中段のプロットは活性化関数の出力($z_i$)を表しています。パラメータ($w_{ij}$)の初期値として正規分布を指定すると、シグモイド関数の出力はそのほとんどが0か1に非常に近い値となっています。シグモイド関数の微分は$\sigma^{'}(x)=\sigma(x)\cdot(1-\sigma(x))$なので、$\sigma(x)$が0や1に近いときは微分値も非常に小さな値となります。
誤差逆伝播の式は
$$
\begin{align}
\delta_{i}^{(k)} &= \sigma^{'}(a_i^{(k)}) \left( \sum_j w_{ij}^{(k+1)} \cdot \delta_{j}^{(k+1)} \right) \\
\frac{\partial E_n}{\partial w_{ij}^{(k)}}  &= \delta_{j}^{(k)} \cdot z_{i}^{(k)}
\end{align}
$$
でした。$\sigma^{'}(a_i^{(k)})$が小さいと後方の層から前方の層に誤差が伝わる際に、値が小さくなってしまいます。

下段のプロットは各層での$\frac{\partial E_n}{\partial w_{ij}^{(k)}}$を表しています。
前方の層(0th layer)は後方の層と比較して分布の絶対値が小さくなっています。

このように誤差が前の層にいくにつれて小さくなるため、前の層が後ろの層と比較して学習が進まなくなります。
この問題は勾配消失の問題として知られています。

勾配消失はパラメータの初期値や、活性化関数を変更することによって解決・緩和することがわかっています。
Kerasの
- [初期化のページ](https://keras.io/initializers/)
- [活性化関数のページ](https://keras.io/activations/)

も参考にしながら、この問題の解決を試みてみましょう。

活性化関数・パラメータの初期化方法の変更はそれぞれコード中の"activation"、"initializer"を変更することによって行えます。

例えばパラメータの初期化を(-0.01, +0.01)の一様分布に変更するときは以下のコードのようにすれば良いです。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import RandomNormal, RandomUniform
from tensorflow.keras.layers import Dense

# データセットの生成
nSamples = 1000
nFeatures = 50
# 100個の入力変数を持つイベント1000個生成。それぞれの入力変数は正規分布に従う
x = np.random.randn(nSamples, nFeatures)
# 正解ラベルは0 or 1でランダムに生成
t = np.random.randint(2, size=nSamples).reshape([nSamples, 1])


# モデルの定義
# 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
model = Sequential()
model.add(Input(shape=(nFeatures,)))
for _ in range(10):
    model.add(
        Dense(
            units=50,
            # 中間層の各ノードで使う活性化関数
            activation="sigmoid",
            # weight(wij)の初期値。ここでは正規分布に従って初期化する
            kernel_initializer=RandomUniform(minval=-0.01, maxval=0.01),  # NOTE: 変更箇所
        )
    )
model.add(
    Dense(
        units=1,
        activation="sigmoid",
        # weight(wij)の初期値。ここでは正規分布に従って初期化する
        kernel_initializer=RandomUniform(minval=-0.01, maxval=0.01),  # NOTE: 変更箇所
    )
)

model.compile(loss="binary_crossentropy", optimizer="adam")

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(3, 1, figsize=(6, 12))

# ウェイト(wij)の初期値をプロット
utils.plot_model_weights(model, ax=ax[0])

# 各ノードの出力(sigma(ai))をプロット
utils.plot_model_hidden_nodes(model, x, ax=ax[1])

# ウェイト(wij)の微分(dE/dwij)をプロット
utils.plot_model_weight_gradients(model, x, t, ax=ax[2])


# 図を表示
plt.show()

この例では活性化関数の出力が0.5付近に集中しています。
どのノードも同じ出力をしているということはノード数を増やした意味があまりなくなっており、多層パーセプトロンの表現力が十分に活かしきれていないことがわかります。
また、勾配消失も先程の例と比較して大きくなっています。